In [40]:
import h5py
import yaml
from nomad.metainfo import Quantity, SubSection
from nomad.datamodel import Schema
from nomad.metainfo import SchemaPackage
from nomad.datamodel import EntryArchive
from nomad.datamodel import ArchiveSection
from nomad.datamodel.hdf5 import HDF5Reference

import numpy as np

In [ ]:
class ScopeFoundryMeasurementData(ArchiveSection):
    name = Quantity(type=str)
    data = Quantity(type=HDF5Reference)

class SFLoggedQuantity(Schema):
    name = Quantity(type=str)
    value = Quantity(type=str)
    unit = Quantity(type=str)

class ScopeFoundryHW(Schema):
    name = Quantity(type=str)
    settings = SubSection(section=SFLoggedQuantity, repeats=True, description='list of configuration for hardware')

class ScopeFoundryMeasurement(Schema):
    name = Quantity(type=str)
    settings = SubSection(section=SFLoggedQuantity, repeats=True, description='list of configuration for measurement')
    datasets = SubSection(section=ScopeFoundryMeasurementData, repeats=True, description='H5 Datasets collected during measurement')

class ScopeFoundryH5(Schema):
    h5_file = Quantity(
        type=str,
        description='HDF5 file',
        a_eln={
            "component": "FileEditQuantity",
        },
    )

    app_name = Quantity(type=str, a_eln={'overview':True}, a_display={'visible': True})
    app_settings = SubSection(section=SFLoggedQuantity, repeats=True, description='list of configuration for app')
    hardware = SubSection(section=ScopeFoundryHW, repeats=True, description='Hardware Components', , a_display={'visible': True})
    measurement = SubSection(section=ScopeFoundryMeasurement, repeats=True, description='Measurements (usually only one per h5)',a_eln={'overview':True}, , a_display={'visible': True})


In [72]:
def create_schema_package():
    return EntryArchive(
        definitions=SchemaPackage(
            name='ScopeFoundry',
            sections=[
                ScopeFoundryH5.m_def,
                ScopeFoundryHW.m_def,
                ScopeFoundryMeasurement.m_def,
                SFLoggedQuantity.m_def,
                ScopeFoundryMeasurementData.m_def
            ]
        )
    )
def save_schema_package_to_yaml():
    with open('scopefoundry_schema_package.archive.yaml', 'wt') as f:
        f.write(yaml.dump(create_schema_package().m_to_dict(with_out_meta=True), indent=2))

save_schema_package_to_yaml()
print(yaml.dump(create_schema_package().m_to_dict(with_out_meta=True), indent=2, sort_keys=False))

definitions:
  name: ScopeFoundry
  section_definitions:
  - name: ScopeFoundryH5
    base_sections:
    - nomad.datamodel.data.EntryData
    quantities:
    - m_annotations:
        eln:
        - component: FileEditQuantity
      name: h5_file
      description: HDF5 file
      type:
        type_kind: python
        type_data: str
    - name: app_name
      type:
        type_kind: python
        type_data: str
    sub_sections:
    - name: app_settings
      description: list of configuration for app
      sub_section: /definitions/section_definitions/3
      repeats: true
    - name: hardware
      description: Hardware Components
      sub_section: /definitions/section_definitions/1
      repeats: true
    - m_annotations:
        eln:
        - overview: true
      name: measurement
      description: Measurements (usually only one per h5)
      sub_section: /definitions/section_definitions/2
      repeats: true
  - name: ScopeFoundryHW
    base_sections:
    - nomad.datamodel.d

In [73]:
import os

def settingsH5_to_NomadLQlist(sH5):
    lqs = []
    for k,v in sH5.attrs.items():
        lq = SFLoggedQuantity(name=k, value=str(v))
        if k in sH5['units'].attrs:
            lq.unit = sH5['units'].attrs[k]
        lqs.append(lq)
    return lqs

def parse(mainfile, archive, logger):
    with h5py.File(mainfile,'r') as H:
        h = archive.data = ScopeFoundryH5(
            h5_file = mainfile,
            app_name = H['app'].attrs['name']
        )
        h.app_settings = settingsH5_to_NomadLQlist(H['app/settings'])
        for HW in H['hardware'].values():
            nHW = ScopeFoundryHW(name=HW.attrs['name'])
            nHW.settings = settingsH5_to_NomadLQlist(HW['settings'])
            h.hardware.append(nHW)
        for M in H['measurement'].values():
            nM = ScopeFoundryMeasurement(name=M.attrs['name'])
            h.measurement.append(nM)
            nM.settings = settingsH5_to_NomadLQlist(M['settings'])
            for name,hData in M.items():
                if not isinstance(hData, h5py.Dataset): continue
                nData = ScopeFoundryMeasurementData(name = name)
                nM.datasets.append(nData)
                nData.data = f'{mainfile}#{hData.name}'
                
            

from nomad.datamodel import EntryMetadata

archive = EntryArchive(metadata=EntryMetadata())
mainfile = 'test1.h5'
parse(mainfile, archive, None)

json_data = archive.m_to_dict()

# Here we replace the schema reference
json_data['data']['m_def'] = \
    '../upload/raw/scopefoundry_schema_package.archive.yaml#/definitions/section_definitions/ScopeFoundryH5'

from IPython.display import JSON

import json
# Save the country as a .archive.json
with open(f'{mainfile}.archive.json', 'wt') as f:
    f.write(json.dumps(json_data, indent=2))

JSON(json_data, indent=4)


<IPython.core.display.JSON object>